# Jet Energy Corrections

## Loading packages and input files

In [ ]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import hist
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
import correctionlib
import copy

plt.style.use(hep.style.CMS)
plt.rcParams.update({
    'font.size': 16,         # Base font size
    'axes.titlesize': 18,    # Title font size
    'axes.labelsize': 16,    # Axis label font size
    'xtick.labelsize': 14,   # X-tick label size
    'ytick.labelsize': 14,   # Y-tick label size
    'legend.title_fontsize': 16, # Legend title size
    'legend.fontsize': 16        # Legend font size
})

In [ ]:
fname = "root://eosuser.cern.ch//eos/user/c/cmsdas/2025/jets/RunIISummer20UL18NanoAODv9/TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2520000/130AF209-596A-BA46-BA9E-D8847511BB0F.root"
events = NanoEventsFactory.from_root(fname, schemaclass=NanoAODSchema.v6).events()

## Exercise 3.1

First, for simplicity, let's apply a simple selection in the jet eta of the jets we are going to study:

In [ ]:
jets = events.Jet[ (np.abs(events.Jet.eta)<2.5) ]

Since the jets in our input files are corrected, we will uncorrect them first. Then, again for simplicity, we will include a jet pt selection:

In [ ]:
uncorr_jets = copy.deepcopy(jets)
uncorr_jets['pt_raw'] = (1 - uncorr_jets['rawFactor']) * uncorr_jets['pt']
uncorr_jets['mass_raw'] = (1 - uncorr_jets['rawFactor']) * uncorr_jets['mass']

uncorr_jets = uncorr_jets[(uncorr_jets.pt_raw > 30)]
events.GenJet = events.GenJet[(events.GenJet.pt > 30)]

Let's make the histograms to compare and the plots:

In [ ]:
hists_uncorr = (
    hist.Hist.new
    .StrCat(["gen", "reco"], name="jet_type")
    .Reg(100, 0, 1000, name="pt", label='jet pt')
    .Reg(40, -5, 5, name="eta", label='jet eta')
    .Weight()
    .fill(
        jet_type='gen',
        pt=ak.flatten(events.GenJet.pt),
        eta=ak.flatten(events.GenJet.eta)
    )
    .fill(
        jet_type='reco',
        pt=ak.flatten(uncorr_jets.pt_raw),
        eta=ak.flatten(uncorr_jets.eta)
    )
)

In [ ]:
gen = hists_uncorr["gen", :, ::sum]
unc_reco = hists_uncorr["reco", :, ::sum]

fig = plt.gcf()
fig.set_size_inches(12, 8)  # Width=12 inches, Height=8 inches
grid = fig.add_gridspec(2, 1, hspace=0, height_ratios=[3, 1])

main_ax = fig.add_subplot(grid[0])
subplot_ax = fig.add_subplot(grid[1], sharex=main_ax)

main, subplot = gen.plot_ratio(unc_reco, rp_num_label="gen", rp_denom_label="uncorr reco", ax_dict={'main_ax' : main_ax, 'ratio_ax': subplot_ax} )
main_ax.set_yscale('log')

### Before and after JECs

Let's looks at some distributions to understand the difference in this correction:

In [ ]:
jerc_file = '/cvmfs/cms.cern.ch/rsync/cms-nanoAOD/jsonpog-integration/POG/JME/2018_UL/jet_jerc.json.gz'
jerc_corr = correctionlib.CorrectionSet.from_file(jerc_file)

uncorr_jets = copy.deepcopy(jets)
uncorr_jets['pt_raw'] = (1 - uncorr_jets['rawFactor']) * uncorr_jets['pt']
uncorr_jets['mass_raw'] = (1 - uncorr_jets['rawFactor']) * uncorr_jets['mass']
uncorr_jets['rho'] = ak.broadcast_arrays(events.fixedGridRhoFastjetAll, uncorr_jets.pt)[0]
j, nj = ak.flatten(uncorr_jets), ak.num(uncorr_jets)

corr = jerc_corr.compound["Summer19UL18_V5_MC_L1L2L3Res_AK4PFchs"]
flat_jec = corr.evaluate( j['area'], j['eta'], j['pt_raw'], j['rho'] )
jec = ak.unflatten( flat_jec, nj )

corr_jets = copy.deepcopy(uncorr_jets)
corr_jets['jet_energy_correction'] = jec
corr_jets['pt'] = corr_jets.pt_raw * jec
corr_jets['mass'] = corr_jets.mass_raw * jec

corr_jets = corr_jets[ (corr_jets.pt > 30) ]

Let's make the histograms to compare genjets with corrected jets:

In [ ]:
hists_corr = (
    hist.Hist.new
    .StrCat(["gen", "reco"], name="jet_type")
    .Reg(100, 0, 1000, name="pt", label='jet pt')
    .Reg(40, -5, 5, name="eta", label='jet eta')
    .Weight()
    .fill(
        jet_type='gen',
        pt=ak.flatten(events.GenJet.pt),
        eta=ak.flatten(events.GenJet.eta)
    )
    .fill(
        jet_type='reco',
        pt=ak.flatten(corr_jets.pt),
        eta=ak.flatten(corr_jets.eta)
    )
)

In [ ]:
gen = hists_corr["gen",:,:].project("pt")
reco = hists_corr["reco",:,:].project("pt")

fig = plt.gcf()
fig.set_size_inches(12, 8)  # Width=12 inches, Height=8 inches
grid = fig.add_gridspec(2, 1, hspace=0, height_ratios=[3, 1])

main_ax = fig.add_subplot(grid[0])
subplot_ax = fig.add_subplot(grid[1], sharex=main_ax)

main, subplot = gen.plot_ratio(reco, rp_num_label="gen", rp_denom_label="reco", ax_dict={'main_ax' : main_ax, 'ratio_ax': subplot_ax} )
main_ax.set_yscale('log')

## Exercise 3.2: JEC Uncertainties

Let's apply the JEC uncertainties (one sigma up and down) to the uncorrected jets:

In [ ]:
uncorr_jets = copy.deepcopy(jets)
uncorr_jets['pt_raw'] = (1 - uncorr_jets['rawFactor']) * uncorr_jets['pt']
uncorr_jets['mass_raw'] = (1 - uncorr_jets['rawFactor']) * uncorr_jets['mass']
uncorr_jets['rho'] = ak.broadcast_arrays(events.fixedGridRhoFastjetAll, uncorr_jets.pt)[0]
j, nj = ak.flatten(uncorr_jets), ak.num(uncorr_jets)

### jet corrections
corr = jerc_corr.compound["Summer19UL18_V5_MC_L1L2L3Res_AK4PFchs"]
flat_jec = corr.evaluate( j['area'], j['eta'], j['pt_raw'], j['rho'] )
jec = ak.unflatten( flat_jec, nj )

### jet corrections uncertainties
corr = jerc_corr["Summer19UL18_V5_MC_Total_AK4PFchs"]
flat_jec_unc = corr.evaluate( j['eta'], j['pt_raw'] )
flat_jec_unc_up = np.ones(len(j)) + flat_jec_unc
flat_jec_unc_down = np.ones(len(j)) - flat_jec_unc

jec_unc_up = ak.unflatten( flat_jec_unc_up, nj )
jec_unc_down = ak.unflatten( flat_jec_unc_down, nj )

corr_jets = copy.deepcopy(uncorr_jets)
corr_jets['jet_energy_correction'] = jec
corr_jets['pt'] = corr_jets.pt_raw * jec
corr_jets['mass'] = corr_jets.mass_raw * jec

corr_jets['pt_unc_up'] = corr_jets.pt * jec_unc_up
corr_jets['pt_unc_down'] = corr_jets.pt * jec_unc_down

corr_jets = corr_jets[ (corr_jets.pt > 30) ]

Let's make the plots to compare:

In [ ]:
hists_unc = (
    hist.Hist.new
    .StrCat(["nominal", "up", 'down'], name="unc")
    .Reg(100, 0, 1000, name="pt", label='jet pt')
    .Weight()
    .fill(
        unc='nominal',
        pt=ak.flatten(corr_jets.pt),
    )
    .fill(
        unc='up',
        pt=ak.flatten(corr_jets.pt_unc_up),
    )
    .fill(
        unc='down',
        pt=ak.flatten(corr_jets.pt_unc_down),
    )
)

In [ ]:
h_nom = hists_unc[{'unc': 'nominal'}]
h_up = hists_unc[{'unc': 'up'}]
h_down = hists_unc[{'unc': 'down'}]

# Setup figure
fig = plt.figure(figsize=(10, 8))
grid = fig.add_gridspec(2, 1, height_ratios=[3, 1], hspace=0)
main_ax = fig.add_subplot(grid[0])
ratio_ax = fig.add_subplot(grid[1], sharex=main_ax)

# Top panel: nominal, up, down
h_nom.plot1d(ax=main_ax, label="Nominal")
h_up.plot1d(ax=main_ax, label="Up")
h_down.plot1d(ax=main_ax, label="Down")
main_ax.set_yscale("log")
main_ax.legend(title="Uncertainty")
main_ax.set_ylabel("Events")

# Bottom panel: up/nom and down/nom
ratio_up = h_up.values() / h_nom.values()
ratio_down = h_down.values() / h_nom.values()
bin_edges = h_nom.axes[0].edges
bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

# Plot ratios
ratio_ax.step(bin_centers, ratio_up, where="mid", label="Up / Nominal", color="orange")
ratio_ax.step(bin_centers, ratio_down, where="mid", label="Down / Nominal", color="green")
ratio_ax.axhline(1.0, color="black", linestyle="--")
ratio_ax.set_ylim(0.5, 1.5)
ratio_ax.set_ylabel("Variation / Nominal")
ratio_ax.set_xlabel(h_nom.axes[0].label)

plt.tight_layout()
plt.show()

## Your turn

You can check different the JEC uncertainties variations for other jet variables

## Exercise 3.3: Jet Energy Resolution

The next cell is the `coffea` implementation to apply jet energy resolution. Notice that this implementation does **not** uses the json-pog but the all txt files. 

This will change in the near future.

In [ ]:
from coffea.jetmet_tools import FactorizedJetCorrector, JetCorrectionUncertainty
from coffea.jetmet_tools import JECStack, CorrectedJetsFactory
from coffea.lookup_tools import extractor

### just in case, let's start from scratch
events = NanoEventsFactory.from_root( fname, schemaclass=NanoAODSchema.v6).events()

ext = extractor()
ext.add_weight_sets([
    "* * ../data/JECs/Summer19UL18_V5_MC/Summer19UL18_V5_MC_L1FastJet_AK4PFchs.txt",
    "* * ../data/JECs/Summer19UL18_V5_MC/Summer19UL18_V5_MC_L2Relative_AK4PFchs.txt",
    "* * ../data/JECs/Summer19UL18_V5_MC/Summer19UL18_V5_MC_L2L3Residual_AK4PFchs.txt",
    "* * ../data/JECs/Summer19UL18_V5_MC/Summer19UL18_V5_MC_L3Absolute_AK4PFchs.txt",
    "* * ../data/JECs/Summer19UL18_V5_MC/Summer19UL18_JRV2_MC_PtResolution_AK4PFchs.txt",
    "* * ../data/JECs/Summer19UL18_V5_MC/Summer19UL18_JRV2_MC_SF_AK4PFchs.txt",
])
ext.finalize()

jec_stack_names = [
    "Summer19UL18_V5_MC_L1FastJet_AK4PFchs",
    "Summer19UL18_V5_MC_L2Relative_AK4PFchs",
    "Summer19UL18_V5_MC_L2L3Residual_AK4PFchs",
    "Summer19UL18_V5_MC_L3Absolute_AK4PFchs",
    "Summer19UL18_JRV2_MC_PtResolution_AK4PFchs",
    "Summer19UL18_JRV2_MC_SF_AK4PFchs"
]

evaluator = ext.make_evaluator()
jec_inputs = {name: evaluator[name] for name in jec_stack_names}
jec_stack = JECStack(jec_inputs)

name_map = jec_stack.blank_name_map
name_map['JetPt'] = 'pt'
name_map['JetMass'] = 'mass'
name_map['JetEta'] = 'eta'
name_map['JetA'] = 'area'

jets = events.Jet

jets['pt_raw'] = (1 - jets['rawFactor']) * jets['pt']
jets['mass_raw'] = (1 - jets['rawFactor']) * jets['mass']
jets['pt_gen'] = ak.values_astype(ak.fill_none(jets.matched_gen.pt, 0), np.float32)
jets['rho'] = ak.broadcast_arrays(events.fixedGridRhoFastjetAll, jets.pt)[0]
name_map['ptGenJet'] = 'pt_gen'
name_map['ptRaw'] = 'pt_raw'
name_map['massRaw'] = 'mass_raw'
name_map['Rho'] = 'rho'

events_cache = events.caches[0]
jet_factory = CorrectedJetsFactory(name_map, jec_stack)
corrected_jets = jet_factory.build(jets, lazy_cache=events_cache)

Keep in mind that JERs needs to be applied to corrected jets. All of this is done behind the scene with the `coffea` `CorrectedJetsFactory`. 

Now let's look at the effect in the jet pt and plot the difference in the JER variations:

In [ ]:
# Build the histogram
hists_jer = (
    hist.Hist.new
    .StrCat(["nominal", "up", 'down'], name="unc", label="JER")
    .Reg(50, 0, 500, name="pt", label='jet pt')
    .Weight()
)

# Fill histograms
hists_jer.fill(unc='nominal', pt=ak.flatten(corrected_jets.pt))
hists_jer.fill(unc='up', pt=ak.flatten(corrected_jets.JER.up.pt))
hists_jer.fill(unc='down', pt=ak.flatten(corrected_jets.JER.down.pt))

# Split the variations
h_nom = hists_jer[{'unc': 'nominal'}]
h_up = hists_jer[{'unc': 'up'}]
h_down = hists_jer[{'unc': 'down'}]

# Prepare figure with two subplots
fig = plt.figure(figsize=(10, 8))
grid = fig.add_gridspec(2, 1, height_ratios=[3, 1], hspace=0)
main_ax = fig.add_subplot(grid[0])
ratio_ax = fig.add_subplot(grid[1], sharex=main_ax)

# --- Top: nominal, up, down ---
h_nom.plot1d(ax=main_ax, label="Nominal")
h_up.plot1d(ax=main_ax, label="JER Up")
h_down.plot1d(ax=main_ax, label="JER Down")
main_ax.set_yscale("log")
main_ax.set_ylabel("Events")
main_ax.legend(title="JER variation")

# --- Bottom: up/nom, down/nom ---
bin_edges = h_nom.axes[0].edges
bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

nom_vals = h_nom.values()
up_vals = h_up.values()
down_vals = h_down.values()

# Avoid division by zero
ratio_up = np.divide(up_vals, nom_vals, out=np.ones_like(up_vals), where=nom_vals != 0)
ratio_down = np.divide(down_vals, nom_vals, out=np.ones_like(down_vals), where=nom_vals != 0)

# Plot ratios
ratio_ax.step(bin_centers, ratio_up, where="mid", label="Up/Nom", color="orange")
ratio_ax.step(bin_centers, ratio_down, where="mid", label="Down/Nom", color="green")
ratio_ax.axhline(1.0, color="black", linestyle="--")
ratio_ax.set_ylim(0.8, 1.2)
ratio_ax.set_ylabel("Ratio")
ratio_ax.set_xlabel(h_nom.axes[0].label)

plt.tight_layout()
plt.show()